In [3]:
#Import Dependencies
import pandas as pd
from sqlalchemy import create_engine

In [4]:
#Extract CSV DataFrame
file = "Resources/top10s.csv"
top_10 = pd.read_csv(file, encoding="Latin-1")
top_10.head()

,Unnamed: 0,title,artist,top genre,year,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop
0,1,"Hey, Soul Sister",Train,neo mellow,2010,97,89,67,-4,8,80,217,19,4,83
1,2,Love The Way You Lie,Eminem,detroit hip hop,2010,87,93,75,-5,52,64,263,24,23,82
2,3,TiK ToK,Kesha,dance pop,2010,120,84,76,-3,29,71,200,10,14,80
3,4,Bad Romance,Lady Gaga,dance pop,2010,119,92,70,-4,8,71,295,0,4,79
4,5,Just the Way You Are,Bruno Mars,pop,2010,109,84,64,-5,9,43,221,2,4,78


In [5]:
#Drop rows before 2015
top_10 = top_10.loc[(top_10['year'] >= 2015)]
top_10.head()

,Unnamed: 0,title,artist,top genre,year,bpm,nrgy,dnce,dB,live,val,dur,acous,spch,pop
268,269,Thinking out Loud,Ed Sheeran,pop,2015,79,45,78,-6,18,59,282,47,3,84
269,270,I'm Not The Only One,Sam Smith,pop,2015,82,49,68,-6,8,49,239,53,4,84
270,271,The Hills,The Weeknd,canadian contemporary r&b,2015,113,56,58,-7,14,14,242,7,5,84
271,272,Love Yourself,Justin Bieber,canadian pop,2015,100,38,61,-10,28,52,234,84,44,83
272,273,Uptown Funk,Mark Ronson,dance pop,2015,115,61,86,-7,3,93,270,1,8,82


In [6]:
#Create new data with selected columns
songs_df = top_10[['title', 'artist', 'dnce', 'live', 'dur']].copy()
songs_df.head()

,title,artist,dnce,live,dur
268,Thinking out Loud,Ed Sheeran,78,18,282
269,I'm Not The Only One,Sam Smith,68,8,239
270,The Hills,The Weeknd,58,14,242
271,Love Yourself,Justin Bieber,61,28,234
272,Uptown Funk,Mark Ronson,86,3,270


In [7]:
# Rename the column headers and reset index
songs_df = songs_df.rename(columns={'title': 'song',
                                   'artist': 'artist',
                                   'dnce': 'dance',
                                   'live': 'live_rec',
                                   'dur': 'duration'})
songs_df = songs_df.reset_index(drop=True)
songs_df.head()

,song,artist,dance,live_rec,duration
0,Thinking out Loud,Ed Sheeran,78,18,282
1,I'm Not The Only One,Sam Smith,68,8,239
2,The Hills,The Weeknd,58,14,242
3,Love Yourself,Justin Bieber,61,28,234
4,Uptown Funk,Mark Ronson,86,3,270


In [8]:
#Extract CSV into DataFrame
file2 = "Resources/Billboard.csv"
billboard = pd.read_csv(file2)
billboard.head()

,Unnamed: 0,Song,Artist,Weeks On #1,Weeks On Chart,Peak Rank
0,0,Blank Space,Taylor Swift,7,36,1
1,1,Take Me To Church,Hozier,0,41,2
2,2,Uptown Funk!,Mark Ronson Featuring Bruno Mars,14,56,1
3,3,Thinking Out Loud,Ed Sheeran,0,58,2
4,4,Lips Are Movin,Meghan Trainor,0,29,4


In [9]:
#Create new data with selected columns
billboard_df = billboard[['Song', 'Artist', 'Weeks On #1', 'Weeks On Chart']].copy()
billboard_df.head()

,Song,Artist,Weeks On #1,Weeks On Chart
0,Blank Space,Taylor Swift,7,36
1,Take Me To Church,Hozier,0,41
2,Uptown Funk!,Mark Ronson Featuring Bruno Mars,14,56
3,Thinking Out Loud,Ed Sheeran,0,58
4,Lips Are Movin,Meghan Trainor,0,29


In [10]:
# Rename the column headers
billboard_df = billboard_df.rename(columns={'Song': 'song',
                                            'Artist': 'artist',
                                            'Weeks On #1': 'weeks_1',
                                   'Weeks On Chart': 'weeks_chart'})
billboard_df.head()

,song,artist,weeks_1,weeks_chart
0,Blank Space,Taylor Swift,7,36
1,Take Me To Church,Hozier,0,41
2,Uptown Funk!,Mark Ronson Featuring Bruno Mars,14,56
3,Thinking Out Loud,Ed Sheeran,0,58
4,Lips Are Movin,Meghan Trainor,0,29


In [11]:
#Create database connection
db_conn = "postgres:password@localhost:5432/etl_db"
engine = create_engine(f'postgresql://{db_conn}') 

In [12]:
#Confirm tables 
engine.table_names()

['billboard', 'spotify']

In [13]:
#Load DataFrames into database
billboard_df.to_sql(name='billboard', con=engine, if_exists='append', index=False)

In [14]:
songs_df.to_sql(name='spotify', con=engine, if_exists='append', index=False)

In [15]:
#Confirm data has been added by quering the spotify table
pd.read_sql_query('select * from spotify', con=engine).head()

,song,artist,dance,live_rec,duration
0,Thinking out Loud,Ed Sheeran,78,18,282
1,I'm Not The Only One,Sam Smith,68,8,239
2,The Hills,The Weeknd,58,14,242
3,Love Yourself,Justin Bieber,61,28,234
4,Uptown Funk,Mark Ronson,86,3,270


In [16]:
#Confirm data has been added by quering the billboard table
pd.read_sql_query('select * from billboard', con=engine).head()

,song,artist,weeks_1,weeks_chart
0,Blank Space,Taylor Swift,7,36
1,Take Me To Church,Hozier,0,41
2,Uptown Funk!,Mark Ronson Featuring Bruno Mars,14,56
3,Thinking Out Loud,Ed Sheeran,0,58
4,Lips Are Movin,Meghan Trainor,0,29
